# LGBM VMD Model development

In [4]:
import pandas as pd
import seaborn as sns
import openpyxl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle

In [5]:
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna
from vmdpy import VMD

In [6]:
from Functions.helper_functions import * 

In [7]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
np.random.seed(42)

# Data

In [9]:
train_wp1 = pd.read_csv('Data/Preprocessing/WP1_train_preprocessed.csv', sep=',')
train_wp2 = pd.read_csv('Data/Preprocessing/WP2_train_preprocessed.csv', sep=',')
train_wp3 = pd.read_csv('Data/Preprocessing/WP3_train_preprocessed.csv', sep=',')
train_wp4 = pd.read_csv('Data/Preprocessing/WP4_train_preprocessed.csv', sep=',')
train_wp5 = pd.read_csv('Data/Preprocessing/WP5_train_preprocessed.csv', sep=',')
train_wp6 = pd.read_csv('Data/Preprocessing/WP6_train_preprocessed.csv', sep=',')

In [10]:
test_wp1 = pd.read_csv('Data/Preprocessing/WP1_test_preprocessed.csv', sep=',')
test_wp2 = pd.read_csv('Data/Preprocessing/WP2_test_preprocessed.csv', sep=',')
test_wp3 = pd.read_csv('Data/Preprocessing/WP3_test_preprocessed.csv', sep=',')
test_wp4 = pd.read_csv('Data/Preprocessing/WP4_test_preprocessed.csv', sep=',')
test_wp5 = pd.read_csv('Data/Preprocessing/WP5_test_preprocessed.csv', sep=',')
test_wp6 = pd.read_csv('Data/Preprocessing/WP6_test_preprocessed.csv', sep=',')
test_dates = pd.read_csv('Data/Initial/test.csv', sep=',').date.values

In [11]:
to_drop = ['date','wd','forecast_time', 'forecast', "forecast_dist"]

In [12]:
u_to_drop = [
    'u_T_1', 'u_T_2', 'u_T_3', 'u_T_4', 'u_T_5', 'u_T_6', 
    'u_T_2_mean', 'u_T_3_mean', 'u_T_4_mean', 'u_T_5_mean', 'u_T_6_mean', 'u_T_7_mean',
    'u_T_8_mean', 'u_T_9_mean', 'u_T_10_mean', 'u_T_11_mean', 'u_T_12_mean','u_T_24_mean',
    'u_T_2_std', 'u_T_4_std', 'u_T_5_std', 'u_T_6_std',
    'u_T_2_median', 'u_T_3_median', 'u_T_4_median', 'u_T_5_median', 'u_T_6_median', 'u_T_12_median','u_T_24_median', 'u_T_36_median',
    'u_T_2_max', 'u_T_3_max', 'u_T_4_max', 'u_T_5_max', 'u_T_6_max', 'u_T_12_max',
    'u_T_2_min', 'u_T_3_min', 'u_T_4_min', 'u_T_5_min', 'u_T_6_min', 'u_T_12_min',
    'u2_T_1', 'u2_T_2', 'u2_T_3', 'u2_T_4', 'u2_T_5', 'u2_T_6', 
    'u2_T_2_mean', 'u2_T_3_mean', 'u2_T_4_mean', 'u2_T_5_mean', 'u2_T_6_mean', 'u2_T_7_mean',
    'u2_T_8_mean', 'u2_T_9_mean', 'u2_T_10_mean', 'u2_T_11_mean', 'u2_T_12_mean','u2_T_24_mean',
    'u2_T_2_std', 'u2_T_4_std', 'u2_T_5_std', 'u2_T_6_std', 'u2_T_24_std',
    'u2_T_2_median', 'u2_T_3_median', 'u2_T_4_median', 'u2_T_5_median', 'u2_T_6_median', 'u2_T_12_median',
    'u2_T_2_max','u2_T_3_max', 'u2_T_4_max','u2_T_5_max', 'u2_T_6_max', 'u2_T_12_max',
    'u2_T_2_min', 'u2_T_3_min', 'u2_T_4_min', 'u2_T_5_min', 'u2_T_6_min',
    'u2_T_12', 'u2_T_36_mean', 'u2_T_36_std', 'u2_T_24_median', 'u2_T_24_max',
    'u_T_36_mean','u_T_12','u_T_24_max','u2_T_36_median','u_T_24_min'
]
ws_to_drop = [
    'ws_T_1', 'ws_T_2', 'ws_T_3', 'ws_T_4', 'ws_T_5', 'ws_T_6', 'ws_T_7', 'ws_T_8', 'ws_T_10','ws_T_11', 'ws_T_12',
    'ws_T_2_mean', 'ws_T_3_mean', 'ws_T_4_mean', 'ws_T_5_mean', 'ws_T_6_mean', 'ws_T_7_mean', 'ws_T_8_mean', 'ws_T_9_mean', 
    'ws_T_10_mean', 'ws_T_11_mean', 'ws_T_12_mean', 'ws_T_24_mean', 
    'ws_T_2_std', 'ws_T_3_std', 'ws_T_4_std', 'ws_T_5_std', 
    'ws_T_2_median', 'ws_T_3_median', 'ws_T_4_median', 'ws_T_5_median', 'ws_T_6_median',
    'ws_T_12_median', 'ws_T_24_median', 'ws_T_36_median',
    'ws_T_2_max', 'ws_T_3_max', 'ws_T_4_max', 'ws_T_5_max','ws_T_6_max', 'ws_T_12_max',
     'ws_T_2_min', 'ws_T_3_min', 'ws_T_4_min', 'ws_T_5_min', 'ws_T_6_min', 'ws_T_12_min','ws_T_24_max','ws_T_24_min'
]

v_to_drop = [
    'v_T_1', 'v_T_2', 'v_T_3', 'v_T_4', 'v_T_5', 'v_T_6', 
    'v_T_2_mean', 'v_T_3_mean', 'v_T_4_mean', 'v_T_5_mean', 'v_T_6_mean', 'v_T_7_mean',
    'v_T_8_mean', 'v_T_9_mean', 'v_T_10_mean', 'v_T_11_mean', 'v_T_12_mean', 'v_T_24_mean','v_T_36_mean',
    'v_T_3_std', 'v_T_4_std', 'v_T_5_std','v_T_6_std','v_T_24_std', 'v_T_36_median',
    'v_T_2_median', 'v_T_3_median', 'v_T_4_median', 'v_T_5_median', 'v_T_6_median', 
    'v_T_2_max', 'v_T_3_max', 'v_T_4_max', 'v_T_5_max', 'v_T_6_max', 'v_T_12_max', 
    'v_T_2_min', 'v_T_3_min', 'v_T_4_min', 'v_T_5_min', 'v_T_6_min', 'v_T_12_min', 
    'v_T_36_min', 'v_T_36', 'v_T_24_max',  'v_T_12_median', 'v_T_24_median',
]

wd_to_drop = [
    'coswd_1', 'coswd_2', 'coswd_3', 'coswd_4', 'coswd_5', 'coswd_6',
    'coswd_2_mean', 'coswd_3_mean', 'coswd_4_mean', 'coswd_5_mean', 'coswd_6_mean', 'coswd_7_mean', 
    'coswd_8_mean', 'coswd_9_mean', 'coswd_10_mean', 'coswd_11_mean', 'coswd_12_mean', 'coswd_24_mean', 
    'coswd_3_std', 'coswd_4_std','coswd_5_std','coswd_2_median', 'coswd_3_median','coswd_4_median', 
    'coswd_5_median', 'coswd_6_median', 'coswd_36_median', 'coswd_24_median', 'coswd_12_median',
    'coswd_2_max', 'coswd_3_max', 'coswd_4_max', 'coswd_5_max', 'coswd_6_max', 'coswd_12_max', 'coswd_24_max',
    'coswd_2_min', 'coswd_3_min', 'coswd_4_min', 'coswd_5_min', 'coswd_6_min', 'coswd_12_min', 'coswd_24_min',
    'ws_T_36_max', 'ws_T_36_min', 'coswd_12', 'coswd_24'
]

other_to_drop = [
    'cos_day', 'u', 'v'
]

feature_corr = u_to_drop+ws_to_drop+v_to_drop+wd_to_drop+other_to_drop
to_drop = feature_corr+to_drop

# LGBM functions

In [13]:
from lightgbm import LGBMRegressor

In [14]:
def create_dataset(data,n):
    n_batch=int(len(data)/84)
    new_data=np.array_split(data,n_batch)
    train = pd.DataFrame()
    val=pd.DataFrame()
    for i in range(0,624):
        if (i+n)%10 ==0:
            val = pd.concat([val,new_data[i]])
        else:
            train=pd.concat([train,new_data[i]])
    return train,val

In [43]:
def hyperparametrization(trial, train_x, test_x, train_y, test_y):
    param = {
        'num_iterations':trial.suggest_int('num_iterations',10,200),
        'metric': 'rmse', 
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.3),
        'max_depth': trial.suggest_int("max_depth", 20, 100),
        'num_leaves' : trial.suggest_int('num_leaves', 2, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    }
    
#     model = LGBMRegressor(**param)  
    model = Pipeline([('scaler', MinMaxScaler()),('ridge', LGBMRegressor(**param))])
#     model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    model.fit(train_x,train_y)
    
    preds = model.predict(test_x)
    
    rmse =  mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [16]:
def vmd(y,k):
    
    #Intrinsic mode generation
     #Empirical Mode Decomposition
    #. some sample parameters for VMD  
    alpha = 1       # moderate bandwidth constraint  
    tau = 0.           # noise-tolerance (no strict fidelity enforcement)  
    K = k              # k modes  
    DC = 0             # no DC part imposed  
    init = 1           # initialize omegas uniformly  
    tol = 1e-7
    u, u_hat, omega = VMD(y,alpha, tau, K, DC, init, tol)
    df_vmfs = pd.DataFrame()
    #Integration in the dataframe
    for num, imf in enumerate(u):
        #print('----Creating VMFwp{0} EMD columns----'.format(num+1))
        df_vmfs['IMFwp{0}'.format(num+1)] = imf
    return df_vmfs

In [36]:
def lgbm_scaled_cross_validation_vmf(X, y1,y2,y3,y4,y_true, params):
   # if params == None:
#         model = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor())])
        #model = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor())])
   # else:
#         model = Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params))])
    model1 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[0]))])
    model2 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[1]))])
    model3 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[2]))])
    model4 = Pipeline([('scaler', RobustScaler()),('ridge', LGBMRegressor(**params[3]))])

    print('-----------LGBM CROSS VALIDATION BEGINNING-----------')
    split = 10
    #kf = KFold(n_splits=split, shuffle=True)       
    lgbm_rmse_scores = []
    lgbm_mae_scores = []
    i = 1
    for n in range(split):
        X_train, X_test = create_dataset(X,n)
        Y1_train, Y1_test = create_dataset(y1,n)
        Y2_train, Y2_test = create_dataset(y2,n)
        Y3_train, Y3_test = create_dataset(y3,n)
        Y4_train, Y4_test = create_dataset(y4,n)
        Y_train, Y_test = create_dataset(y_true,n)

        model1.fit(X_train, Y1_train)
        model2.fit(X_train, Y2_train)
        model3.fit(X_train, Y3_train)
        model4.fit(X_train, Y4_train)

        prediction = model1.predict(X_test)+model2.predict(X_test)+model3.predict(X_test)+model4.predict(X_test)
        lgbm_rmse_scores.append(mean_squared_error(Y_test, prediction,squared=False))
        lgbm_mae_scores.append(mean_absolute_error(Y_test, prediction))
        
        print(show_evaluation(prediction, Y_test))
        print(f'-------------------FOLD {i}-----------------')
        i+=1

    print('---------------CROSS VALIDATION COMPLETE-------------')
    print('--------------------------RMSE-----------------------')
    display_scores(lgbm_rmse_scores)
    print('--------------------------MAE------------------------')
    display_scores(lgbm_mae_scores)

## WP1 

| |  | Mean | Std | Sum up |
| --- | --- | --- | --- | --- |
| No params | RMSE | 0.10344875448880764 | 0.0019070131550065564 |  |
| No params - StandardScaler | RMSE | 0.10355855436437653 | 0.0019043784372002228 |  |
| After tuning 50trials| RMSE | 0.06830085723562579 | 0.0012998671387256361 | To keep, maybe redo optuna with warm start with it - first sub|
| After tuning 100trials| RMSE | 0.0695930431604128 | 0.0015123892627707553 | |
| After tuning W.S 60trials | RMSE | 0.0671868765178121 | 0.0019357892229500213 | second sub - BETTER |
| After tuning W.S 50trials - StandardScaler | RMSE | 0.06705406618054667 | 0.0013509038914922506 | with warm start 1 parameters |
| After tuning W.S 50trials - RobustScaler | RMSE | 0.06736753027260156 | 0.0009971394767292064 | |
| --- | --- | --- | --- | --- |
| No params | MAE | 0.07306057409517844 | 0.0009857628815465762 |  |
| No params - StandardScaler | MAE | 0.07310137331348271 | 0.0010945145617042643 |  |
| After tuning 50trials| MAE | 0.04481696427654311 | 0.000727722171899004 |  |
| After tuning 100trials| MAE | 0.045696725242359994 | 0.0008508558100930331 |  |
| After tuning W.S 60trials | MAE | 0.04322865305311156 | 0.0009454253743042544 | |
| After tuning W.S 50trials - StandardScaler | MAE | 0.043173032898780175 | 0.0007696056550296055 | with warm start 1 parameters |
| After tuning W.S 50trials - RobustScaler | MAE | 0.043246198690701294 | 0.000596545751975941 | |

In [18]:
wp1_X = train_wp1[[c for c in train_wp1 if c not in ["wp"]] + ["wp"]].drop(to_drop+feature_corr, axis = 1)
X1 = wp1_X.drop('wp', axis=1)
y1 = wp1_X['wp']

In [19]:
vmf_1 = vmd(y1,4)

In [ ]:
params_1=[]
for i in range(1,5):
    def objective_wp1(trial,data=X1,target=vmf_1['IMFwp'+str(i)]):
        train_x, test_x = create_dataset(data,0)
        train_y,test_y=create_dataset(target,0)
        return hyperparametrization(trial, train_x, test_x, train_y, test_y)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_wp1, n_trials=50)
    #write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
    params_1.append(study.best_trial.params)
    #best_trial

In [67]:
#optuna.logging.set_verbosity(0)

In [65]:
optuna.logging.get_verbosity()

2

In [93]:
params_1

[]

In [49]:
#def objective_wp1(trial,data=X1,target=vmf_1['IMFwp1']):
#    train_x, test_x = create_dataset(data,0)
#    train_y,test_y=create_dataset(target,0)
#    return hyperparametrization(trial, train_x, test_x, train_y, test_y)

In [50]:
#study = optuna.create_study(direction='minimize')
#study.optimize(objective_wp1, n_trials=30)
#write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
#best_trial = study.best_trial.params
#best_trial

In [47]:
#30
#param_1_1 = {
#        'reg_alpha': 0.005623861124729257,
 #  'reg_lambda': 0.22406174588522088,
  #  'colsample_bytree': 0.7,
 #'subsample': 0.4,
 #'learning_rate': 0.11907721244377018,
 #'max_depth': 31,
 #'num_leaves': 776,
 #'min_child_samples': 29}
#
#param_1_2 = {'reg_alpha': 0.003075982697697176,
 #'reg_lambda': 9.876348984550155,
 #'colsample_bytree': 0.3,
 #'subsample': 0.7,
 #'learning_rate': 0.15883690225856262,
 #'max_depth': 64,
 #'num_leaves': 657,
 #'min_child_samples': 20}
#
#param_1_3 = {'reg_alpha': 0.018182754684069484,
# 'reg_lambda': 0.006831890665560452,
# 'colsample_bytree': 0.5,
# 'subsample': 0.5,
# 'learning_rate': 0.11202987530753453,
# 'max_depth': 53,
# 'num_leaves': 848,
# 'min_child_samples': 27}
    
#param_1_4 = {'reg_alpha': 0.0038530142853001356,
# 'reg_lambda': 0.0021589196685704304,
# 'colsample_bytree': 1.0,
# 'subsample': 0.6,
# 'learning_rate': 0.05089100428869571,
# 'max_depth': 94,
# 'num_leaves': 881,
# 'min_child_samples': 29}

#params_vmf=[param_1_1,param_1_2,param_1_3,param_1_4]

In [48]:
#y1_vmfs = []
#for i in range(1,5):
#    y1_vmfs.append(vmf_1['IMFwp'+str(i)])
#y_vmfs

In [46]:
lgbm_scaled_cross_validation_vmf(X1, vmf_1['IMFwp1'],vmf_1['IMFwp2'],vmf_1['IMFwp3'],vmf_1['IMFwp4'],y1, params_1)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.09278747975585935
MAE score: 0.06453001424424233
None
-------------------FOLD 1-----------------
RMSE score: 0.10298803883287531
MAE score: 0.06942072886164485
None
-------------------FOLD 2-----------------
RMSE score: 0.09811999866968998
MAE score: 0.0700667543726276
None
-------------------FOLD 3-----------------
RMSE score: 0.10727334167300454
MAE score: 0.07191095912120853
None
-------------------FOLD 4-----------------
RMSE score: 0.107423144610324
MAE score: 0.07211693548470784
None
-------------------FOLD 5-----------------
RMSE score: 0.09773209716044177
MAE score: 0.06607837756164867
None
-------------------FOLD 6-----------------
RMSE score: 0.09237100146931441
MAE score: 0.06310940092591404
None
-------------------FOLD 7-----------------
RMSE score: 0.09628197628562973
MAE score: 0.06521873046767863
None
-------------------FOLD 8-----------------
RMSE score: 0.09799794976714232
MAE score: 0.0669083148629056

In [51]:
# lgbm_cross_validation(X1, y1, params_1)
#lgbm_scaled_cross_validation(X1, y1, params_1)

## WP2

| |  | Mean | Std | Sum up|
| --- | --- | --- | --- | |
| No params | RMSE | 0.10935335541057582 | 0.0014425096116734836 | |
| No params - StandardScaler | RMSE | 0.10938240918068962 | 0.002563571983412213 |  |
| After tuning - 50trials| RMSE | 0.0725081520968898 | 0.0016974702626377217 | |
| After tuning 100trials| RMSE | 0.0707064364904941 | 0.001396820290618349 | More stable, to keep |
| After tuning W.S. 50trials - StandardScaler | RMSE | 0.07072007438762447 | 0.0010396359382112443 | With 100trials parameters |
| After tuning W.S. 50trials - RobustScaler | RMSE | 0.07014346508497502 | 0.0013671602187583412 | |
| 50 trials per vmf - RobustScaler | RMSE | 0.06362570142793085 |0.001287144743120583 | |
| --- | --- | --- | --- |---|
| No params | MAE | 0.07681923856705511 | 0.0008670825615244791 | |
| No params - StandardScaler | MAE | 0.0768829843356125 | 0.0017568758903956435 |  |
| After tuning - 50trials| MAE | 0.04512164110351975 |  0.0006579433030966575 | |
| After tuning 100trials| MAE | 0.04457902842458915 | 0.0006807155447311589 | |
| After tuning W.S. 50trials - StandardScaler | MAE | 0.04456161763710905 | 0.0005342365137369251 | With 100trials parameters |
| After tuning W.S. 50trials - RobustScaler | MAE | 0.044471464183373825 | 0.0007098731360466324 | |
| 50 trials per vmf - RobustScaler | MAE | 0.04180182904890137 |0.000707244027895384 | |

In [37]:
wp2_X = train_wp2[[c for c in train_wp2 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X2 = wp2_X.drop('wp', axis=1)
y2 = wp2_X['wp']

In [38]:
# lgbm_cross_validation(X2, y2, None)
#lgbm_scaled_cross_validation(X2, y2, None)

In [39]:
vmf_2=vmd(y2,4)

In [ ]:
params_2=[]
for i in range(1,5):
    def objective_wp2(trial,data=X2,target=vmf_2['IMFwp'+str(i)]):
        train_x, test_x = create_dataset(data,0)
        train_y,test_y=create_dataset(target,0)
        return hyperparametrization(trial, train_x, test_x, train_y, test_y)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_wp2, n_trials=30)
    #write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
    params_2.append(study.best_trial.params)
    #best_trial

In [41]:
params_2

[{'reg_alpha': 0.8989626290992829,
  'reg_lambda': 0.02860416768666997,
  'colsample_bytree': 0.9,
  'subsample': 0.7,
  'learning_rate': 0.14818986270396325,
  'max_depth': 77,
  'num_leaves': 493,
  'min_child_samples': 139},
 {'reg_alpha': 0.04814562073250691,
  'reg_lambda': 0.1853132756141036,
  'colsample_bytree': 0.7,
  'subsample': 0.6,
  'learning_rate': 0.12216486361334354,
  'max_depth': 46,
  'num_leaves': 451,
  'min_child_samples': 2},
 {'reg_alpha': 0.13886086557652427,
  'reg_lambda': 0.006035145058825088,
  'colsample_bytree': 0.5,
  'subsample': 0.8,
  'learning_rate': 0.11292949949226556,
  'max_depth': 80,
  'num_leaves': 408,
  'min_child_samples': 28},
 {'reg_alpha': 0.03208743299589346,
  'reg_lambda': 0.05996184640295221,
  'colsample_bytree': 0.6,
  'subsample': 0.8,
  'learning_rate': 0.09630331232627548,
  'max_depth': 91,
  'num_leaves': 351,
  'min_child_samples': 34}]

In [42]:
lgbm_scaled_cross_validation_vmf(X2, vmf_2['IMFwp1'],vmf_2['IMFwp2'],vmf_2['IMFwp3'],vmf_2['IMFwp4'],y2, params_2)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.10038003588674906
MAE score: 0.06875916314897694
None
-------------------FOLD 1-----------------
RMSE score: 0.1204145323195303
MAE score: 0.08254122024341674
None
-------------------FOLD 2-----------------
RMSE score: 0.10945412554541814
MAE score: 0.0756417143528146
None
-------------------FOLD 3-----------------
RMSE score: 0.10530271725132866
MAE score: 0.07082110354262629
None
-------------------FOLD 4-----------------
RMSE score: 0.11349410338125623
MAE score: 0.07462548012578445
None
-------------------FOLD 5-----------------
RMSE score: 0.10306035942620158
MAE score: 0.06878288458227325
None
-------------------FOLD 6-----------------
RMSE score: 0.10739607999279667
MAE score: 0.07356228035706461
None
-------------------FOLD 7-----------------
RMSE score: 0.11611966567312369
MAE score: 0.07698641615965333
None
-------------------FOLD 8-----------------
RMSE score: 0.10216120078531413
MAE score: 0.069284513201653

## WP3

| |  | Mean | Std ||
| --- | --- | --- | --- ||
| No params | RMSE | 0.10392558077951244 | 0.0019038044796542812 ||
| No params - StandardScaler | RMSE | 0.10354114984428979 | 0.0013629826554706927 ||
| After tuning - 50trials| RMSE | 0.058253804820626545 | 0.0009893279354834155 | More stable, to keep |
| After tuning - 100trials| RMSE | 0.058338944346627106 | 0.0017133930174837203 ||
| After tuning W.S. - 50trials - StandardScaler | RMSE | 0.05839355310487706 | 0.0009599824558874801 ||
| After tuning - 50trial - StandardScaler | RMSE | 0.05828701989178382 | 0.0013469437248627486 | with 50trials best|
| After tuning - 50trial W.S. - RobustScaler | RMSE | 0.05907568414783913 | 0.0010120672015320588 | |
| 50 trials per vmf - RobustScaler | RMSE | 0.055222505478173776 |0.001617631570145444 | |
| --- | --- | --- | --- | --- |
| No params | MAE | 0.07550802464973318 | 0.0012006073434917633 ||
| No params - StandardScaler | MAE | 0.0753220326933334 | 0.0007353049410651828 | |
| After tuning - 50trials| MAE | 0.03787310900962521 | 0.000442034368456366 ||
| After tuning - 100trials| MAE | 0.03838030476025398 | 0.0007480100565996748 ||
| After tuning W.S. - 50trials -StandardScaler | MAE | 0.03838277636708219 | 0.0006195357541130345 ||
| After tuning - 50trials - StandardScaler | MAE | 0.03796602386426936 | 0.0006588818520419181 | with 50trials best|
| After tuning - 50trial W.S. - RobustScaler | MAE | 0.03824961618901729 | 0.0007043585347168141 | |
| 50 trials per vmf - RobustScaler | MAE | 0.03743940912757508 |0.0007763774066394942 | |


In [68]:
wp3_X = train_wp3[[c for c in train_wp3 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X3 = wp3_X.drop('wp', axis = 1)
y3 = wp3_X['wp']


In [69]:
vmf_3=vmd(y3,4)

In [71]:
params_3=[]
for i in range(1,5):
    def objective_wp3(trial,data=X3,target=vmf_3['IMFwp'+str(i)]):
        train_x, test_x = create_dataset(data,0)
        train_y,test_y=create_dataset(target,0)
        return hyperparametrization(trial, train_x, test_x, train_y, test_y)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_wp3, n_trials=50)
    #write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
    params_3.append(study.best_trial.params)
    #best_trial

In [72]:
params_3

[{'num_iterations': 198,
  'reg_alpha': 0.013604579665063635,
  'reg_lambda': 0.001104195569705583,
  'colsample_bytree': 0.5,
  'subsample': 1.0,
  'learning_rate': 0.03570342942079682,
  'max_depth': 77,
  'num_leaves': 545,
  'min_child_samples': 35},
 {'num_iterations': 146,
  'reg_alpha': 0.017893652000519254,
  'reg_lambda': 0.03568880524513762,
  'colsample_bytree': 0.6,
  'subsample': 0.7,
  'learning_rate': 0.057595558537054746,
  'max_depth': 57,
  'num_leaves': 887,
  'min_child_samples': 16},
 {'num_iterations': 194,
  'reg_alpha': 0.0011405663569969192,
  'reg_lambda': 0.03576227430274568,
  'colsample_bytree': 0.6,
  'subsample': 0.5,
  'learning_rate': 0.06393047967439963,
  'max_depth': 94,
  'num_leaves': 374,
  'min_child_samples': 54},
 {'num_iterations': 173,
  'reg_alpha': 0.001997020726371583,
  'reg_lambda': 0.938978270132322,
  'colsample_bytree': 0.9,
  'subsample': 0.5,
  'learning_rate': 0.11345836510280095,
  'max_depth': 24,
  'num_leaves': 715,
  'min_chil

In [73]:
lgbm_scaled_cross_validation_vmf(X3, vmf_3['IMFwp1'],vmf_3['IMFwp2'],vmf_3['IMFwp3'],vmf_3['IMFwp4'],y3, params_3)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.1060574269919031
MAE score: 0.07281683676066975
None
-------------------FOLD 1-----------------
RMSE score: 0.10818762021762672
MAE score: 0.07296957337232776
None
-------------------FOLD 2-----------------
RMSE score: 0.09861201829186156
MAE score: 0.06811066522355663
None
-------------------FOLD 3-----------------
RMSE score: 0.09212888035222459
MAE score: 0.06459443912699175
None
-------------------FOLD 4-----------------
RMSE score: 0.08651070645554249
MAE score: 0.06103123198771605
None
-------------------FOLD 5-----------------
RMSE score: 0.08920883846824865
MAE score: 0.06196663099739528
None
-------------------FOLD 6-----------------
RMSE score: 0.09195705948463435
MAE score: 0.06351992392362356
None
-------------------FOLD 7-----------------
RMSE score: 0.10207941847652546
MAE score: 0.06914274987184395
None
-------------------FOLD 8-----------------
RMSE score: 0.09970201185588567
MAE score: 0.06711069611446

## WP4

| |  | Mean | Std | |
| --- | --- | --- | --- | --- |
| No params | RMSE | 0.10486204816363351 | 0.0015105949978751166 ||
| No params  - Std | RMSE | 0.10480849342496516 | 0.0010370863436755212 ||
| After tuning - 50trials| RMSE | 0.06513233717204232 | 0.0015891617240032727 ||
| After tuning 100trials| RMSE | 0.06357594848470964 | 0.0013676749030776929 ||
| After tuning with W.S - 50trials - Std | RMSE | 0.06339401569270936 | 0.001228053306037005 ||
| After tuning with W.S. - 50trials - RobustScaler | RMSE | 0.0632324115841705 | 0.0010081050240456021 ||
| 50 trials per vmf - RobustScaler | RMSE |0.0593103048767063  |0.0011712884174825636 | |
| --- | --- | --- | --- | --- |
| No params | MAE | 0.07564776733421566 | 0.00104638869825841 ||
| No params  - Std | MAE | 0.07570794104041156 | 0.0008419207475550308 ||
| After tuning - 50trials| MAE | 0.04219236028055372 | 0.0008190579419060266 ||
| After tuning 100trials| MAE |0.04172111697148837  | 0.0009349285385250968 ||
| After tuning with W.S - 50trials  - Std | MAE | 0.04150668920859586 | 0.0005729825500890684 ||
| After tuning with W.S. - 50trials - RobustScaler | MAE | 0.04170428506837879 | 0.0006101247158768171 ||
| 50 trials per vmf - RobustScaler | MAE | 0.04115807160847805 |0.0006471350512019214 | |

In [74]:
wp4_X = train_wp4[[c for c in train_wp4 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X4 = wp4_X.drop('wp', axis = 1)
y4 = wp4_X['wp']

In [75]:
vmf_4 = vmd(y4,4)

In [76]:
params_4=[]
for i in range(1,5):
    def objective_wp4(trial,data=X4,target=vmf_4['IMFwp'+str(i)]):
        train_x, test_x = create_dataset(data,0)
        train_y,test_y=create_dataset(target,0)
        return hyperparametrization(trial, train_x, test_x, train_y, test_y)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_wp4, n_trials=50)
    #write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
    params_4.append(study.best_trial.params)
    #best_trial

In [77]:
params_4

[{'num_iterations': 190,
  'reg_alpha': 0.08740832339553317,
  'reg_lambda': 0.05234226447415364,
  'colsample_bytree': 0.6,
  'subsample': 0.5,
  'learning_rate': 0.05511212634467807,
  'max_depth': 65,
  'num_leaves': 953,
  'min_child_samples': 78},
 {'num_iterations': 168,
  'reg_alpha': 0.004422533925075392,
  'reg_lambda': 0.02532810814326587,
  'colsample_bytree': 0.8,
  'subsample': 0.8,
  'learning_rate': 0.08891098092291012,
  'max_depth': 93,
  'num_leaves': 741,
  'min_child_samples': 16},
 {'num_iterations': 187,
  'reg_alpha': 0.0014580334405515133,
  'reg_lambda': 0.16466463497716385,
  'colsample_bytree': 0.9,
  'subsample': 0.6,
  'learning_rate': 0.09659577021871608,
  'max_depth': 85,
  'num_leaves': 372,
  'min_child_samples': 32},
 {'num_iterations': 147,
  'reg_alpha': 0.005340968904620406,
  'reg_lambda': 0.00482525231455719,
  'colsample_bytree': 0.7,
  'subsample': 0.4,
  'learning_rate': 0.16200076449653428,
  'max_depth': 72,
  'num_leaves': 305,
  'min_child

In [78]:
lgbm_scaled_cross_validation_vmf(X4, vmf_4['IMFwp1'],vmf_4['IMFwp2'],vmf_4['IMFwp3'],vmf_4['IMFwp4'],y4, params_4)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.08891207264274348
MAE score: 0.06381650696113399
None
-------------------FOLD 1-----------------
RMSE score: 0.09654557832557174
MAE score: 0.06777196143168283
None
-------------------FOLD 2-----------------
RMSE score: 0.09918889459646066
MAE score: 0.06925798489665161
None
-------------------FOLD 3-----------------
RMSE score: 0.1024441653741253
MAE score: 0.06999534705713298
None
-------------------FOLD 4-----------------
RMSE score: 0.1002010995683427
MAE score: 0.06781431417666353
None
-------------------FOLD 5-----------------
RMSE score: 0.09816107525473769
MAE score: 0.06519721917437178
None
-------------------FOLD 6-----------------
RMSE score: 0.08977182599067389
MAE score: 0.06296171758847799
None
-------------------FOLD 7-----------------
RMSE score: 0.09821707531675966
MAE score: 0.06680710435668857
None
-------------------FOLD 8-----------------
RMSE score: 0.09885085770138041
MAE score: 0.066265496038151

## WP5

| |  | Mean | Std |
| --- | --- | --- | --- |
| No params | RMSE | 0.11722129743692011 | 0.0017732599261516583 |
| No params - Std | RMSE | 0.11729790317307003 | 0.0014884067903823003 |
| After tuning - 50trials| RMSE | 0.07721413638593042 | 0.0011020420293213135 |
| After tuning - 100trials| RMSE | 0.07297648991888442 | 0.0014970317509404526 |
| After tuning - 100trials - warm start | RMSE | 0.07362803793800192 | 0.0013223501622953715 |
| After tuning 50trials - std - warm start | RMSE | 0.07379631452164911 | 0.0019249511778190373 |
| After tuning 50trials W.S - RobustScaler | RMSE | 0.07295312559507504 | 0.0017094978756410563 |
| 50 trials per vmf | RMSE | 0.06800612721907516 | 0.001364694216955839 |
| --- | --- | --- | --- |
| No params | MAE | 0.08497074568090211 | 0.0009101526501392155 |
| No params - Std | MAE | 0.0849648687365363 | 0.0011901882563545429 |
| After tuning - 50trials| MAE | 0.051677856581467195 | 0.0006374939894477714 |
| After tuning - 100trials| MAE | 0.04765271414503236 | 0.0006257356756510128 |
| After tuning - 100trials - warm trials | MAE | 0.04785179154681675 | 0.0005795839605605526 |
| After tuning 50trials - std - warm start | MAE | 0.0480147138609328 | 0.0008230908433814974 |
| After tuning 50trials W.S - RobustScaler | MAE | 0.04783683731205745 | 0.0007485320718195094 |
| 50 trials per vmf | MAE | 0.04587332776201404 | 0.0008842958670279714 |

In [79]:
wp5_X = train_wp5[[c for c in train_wp5 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X5 = wp5_X.drop('wp', axis = 1)
y5 = wp5_X['wp']

In [80]:
vmf_5=vmd(y5,4)

In [81]:
params_5=[]
for i in range(1,5):
    def objective_wp5(trial,data=X5,target=vmf_5['IMFwp'+str(i)]):
        train_x, test_x = create_dataset(data,0)
        train_y,test_y=create_dataset(target,0)
        return hyperparametrization(trial, train_x, test_x, train_y, test_y)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_wp5, n_trials=50)
    #write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
    params_5.append(study.best_trial.params)
    #best_trial

In [82]:
params_5

[{'num_iterations': 184,
  'reg_alpha': 0.06489604162697013,
  'reg_lambda': 0.055007646697995204,
  'colsample_bytree': 0.9,
  'subsample': 0.4,
  'learning_rate': 0.11026263866224266,
  'max_depth': 21,
  'num_leaves': 331,
  'min_child_samples': 178},
 {'num_iterations': 164,
  'reg_alpha': 0.005409667198241994,
  'reg_lambda': 0.005321378546562433,
  'colsample_bytree': 0.9,
  'subsample': 0.6,
  'learning_rate': 0.07753174946391972,
  'max_depth': 44,
  'num_leaves': 554,
  'min_child_samples': 19},
 {'num_iterations': 151,
  'reg_alpha': 0.9371384169028797,
  'reg_lambda': 0.022972260326287724,
  'colsample_bytree': 0.7,
  'subsample': 0.5,
  'learning_rate': 0.1506752607060971,
  'max_depth': 99,
  'num_leaves': 805,
  'min_child_samples': 81},
 {'num_iterations': 126,
  'reg_alpha': 0.0023359999509967723,
  'reg_lambda': 1.0482967133747603,
  'colsample_bytree': 0.5,
  'subsample': 1.0,
  'learning_rate': 0.12696480211679992,
  'max_depth': 90,
  'num_leaves': 575,
  'min_child

In [83]:
lgbm_scaled_cross_validation_vmf(X5, vmf_5['IMFwp1'],vmf_5['IMFwp2'],vmf_5['IMFwp3'],vmf_5['IMFwp4'],y5, params_5)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.10207058891023578
MAE score: 0.0713443866487568
None
-------------------FOLD 1-----------------
RMSE score: 0.10599291612407934
MAE score: 0.07257188473917432
None
-------------------FOLD 2-----------------
RMSE score: 0.11459649722375616
MAE score: 0.07801326522204596
None
-------------------FOLD 3-----------------
RMSE score: 0.11209855774372034
MAE score: 0.07879658055735482
None
-------------------FOLD 4-----------------
RMSE score: 0.12473622318296942
MAE score: 0.08696070626115071
None
-------------------FOLD 5-----------------
RMSE score: 0.11368688554270887
MAE score: 0.07819949803301844
None
-------------------FOLD 6-----------------
RMSE score: 0.10620772359216918
MAE score: 0.07432056535288271
None
-------------------FOLD 7-----------------
RMSE score: 0.11493016175266649
MAE score: 0.07835202711086822
None
-------------------FOLD 8-----------------
RMSE score: 0.11268304967455353
MAE score: 0.07614615451261

## WP6

| |  | Mean | Std |
| --- | --- | --- | --- |
| No params | RMSE | 0.0940394026188472 | 0.0010749562915831372 |
| No params - std | RMSE | 0.09409110695713666 | 0.0013476052174559326 |
| After tuning - 50trials| RMSE | 0.05404362835213171 | 0.0008595325139047733 |
| After tuning 100trials| RMSE | 0.054861488499908594 | 0.0007335378238383901 |
| After tuning 50trials std - W.S. | RMSE |  0.05446642457662869 | 0.0009850877593637966 |
| After tuning with W.S. - 50trials - RobustScaler | RMSE | 0.053430744425113176 | 0.0010865698281516414 ||
| 50 trials per vmf | RMSE | 0.05055279132726877 | 0.0006313072455945034 |
| --- | --- | --- | --- |
| No params | MAE | 0.070455643271004 | 0.0006641538274191148 |
| After tuning - 50trials| MAE | 0.03657758274248596 | 0.0005325521314198646 |
| After tuning 100trials| MAE | 0.03783933495157941 | 0.00045956939815828987 |
| After tuning 50trials std - warm start| MAE | 0.0367847898055025 | 0.00039001216012464674 |
| After tuning with W.S. - 50trials - RobustScaler | RMSE | 0.03652663713189234 | 0.0006251342721346248 ||
| 50 trials per vmf | MAE | 0.0357185614133554 | 0.00028295806558658535 |

In [84]:
wp6_X = train_wp6[[c for c in train_wp6 if c not in ["wp"]] + ["wp"]].drop(to_drop, axis = 1)
X6 = wp6_X.drop('wp', axis = 1)
y6 = wp6_X['wp']


In [85]:
vmf_6=vmd(y6,4)

In [86]:
params_6=[]
for i in range(1,5):
    def objective_wp6(trial,data=X6,target=vmf_6['IMFwp'+str(i)]):
        train_x, test_x = create_dataset(data,0)
        train_y,test_y=create_dataset(target,0)
        return hyperparametrization(trial, train_x, test_x, train_y, test_y)
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_wp6, n_trials=50)
    #write_results('Data/Hyperparametrization/lgbm_vmd_50trials_cv.xlsx', 'wp1', study.trials_dataframe())
    params_6.append(study.best_trial.params)
    #best_trial

In [87]:
params_6

[{'num_iterations': 200,
  'reg_alpha': 0.004401912791258752,
  'reg_lambda': 3.5836797388523163,
  'colsample_bytree': 0.9,
  'subsample': 0.4,
  'learning_rate': 0.09929313022106498,
  'max_depth': 85,
  'num_leaves': 618,
  'min_child_samples': 134},
 {'num_iterations': 198,
  'reg_alpha': 0.004851715505257754,
  'reg_lambda': 0.033387475224379924,
  'colsample_bytree': 0.5,
  'subsample': 0.6,
  'learning_rate': 0.06866408396138152,
  'max_depth': 100,
  'num_leaves': 752,
  'min_child_samples': 6},
 {'num_iterations': 194,
  'reg_alpha': 0.015448045192685167,
  'reg_lambda': 0.06754565730564002,
  'colsample_bytree': 0.8,
  'subsample': 0.7,
  'learning_rate': 0.03440941519325138,
  'max_depth': 86,
  'num_leaves': 504,
  'min_child_samples': 20},
 {'num_iterations': 135,
  'reg_alpha': 0.04292869814058017,
  'reg_lambda': 0.0016107561795408506,
  'colsample_bytree': 0.7,
  'subsample': 0.8,
  'learning_rate': 0.06367284713708012,
  'max_depth': 68,
  'num_leaves': 664,
  'min_chi

In [88]:
lgbm_scaled_cross_validation_vmf(X6, vmf_6['IMFwp1'],vmf_6['IMFwp2'],vmf_6['IMFwp3'],vmf_6['IMFwp4'],y6, params_6)

-----------LGBM CROSS VALIDATION BEGINNING-----------
RMSE score: 0.07723576297315457
MAE score: 0.05626532521463109
None
-------------------FOLD 1-----------------
RMSE score: 0.08391360524292042
MAE score: 0.06027632654303539
None
-------------------FOLD 2-----------------
RMSE score: 0.08957150818348422
MAE score: 0.06328572376287407
None
-------------------FOLD 3-----------------
RMSE score: 0.08652882027755264
MAE score: 0.06167631641106222
None
-------------------FOLD 4-----------------
RMSE score: 0.09105409987030634
MAE score: 0.06277940989940647
None
-------------------FOLD 5-----------------
RMSE score: 0.08878002632316084
MAE score: 0.06137593176491158
None
-------------------FOLD 6-----------------
RMSE score: 0.08245386569803609
MAE score: 0.0605293261446593
None
-------------------FOLD 7-----------------
RMSE score: 0.08690182464304116
MAE score: 0.06094273867761041
None
-------------------FOLD 8-----------------
RMSE score: 0.08332078320626667
MAE score: 0.05850781293988

# Predictions

## Functions

In [89]:
to_drop_test = ['date','wd','forecast_time', 'forecast', "forecast_dist", 'wp']+feature_corr
def make_prediction_dataset(test, to_drop=to_drop_test):
    test_to_predict = test.dropna(subset=['ws','u','v'], how = 'any') # keeps only lines with u,v,ws,wd
    test_to_predict = test_to_predict[test_to_predict['wp'].isna()] # keeps only lines with no wp
    test_to_predict = test_to_predict.sort_values(by=['date', 'forecast_time'], ascending = [True, False]).drop_duplicates(subset='date')
    test_to_predict = test_to_predict.drop(to_drop, axis = 1)
    return test_to_predict

In [90]:
def make_submission_file(lst_X_trains, lst_y_trains,lst_y_vmfs,lst_tests, lst_models, dates):
    i = 1
    lst_prediction = []
    lst_models_trained = []
    for X, y, test, model in zip(lst_X_trains, lst_y_vmfs, lst_tests, lst_models):
        print(f'--------------Model {i}--------------')
        model[0].fit(X, y[0])
        model[1].fit(X, y[1])
        model[2].fit(X, y[2])
        model[3].fit(X, y[3])
        print(f'True:\n\tMin:{min(lst_y_trains[i-1])}\n\tMax:{max(lst_y_trains[i-1])}\n\tMean:{lst_y_trains[i-1].mean()}')
        predictions = (model[0].predict(test)+
                       model[1].predict(test)+
                       model[2].predict(test)+
                       model[3].predict(test))
        print(f'Prediction:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
#         predictions = [min(y) if i < 0 else i for i in predictions]
#         predictions = [max(y) if i > max(y) else i for i in predictions]
        predictions = [0 if i < 0 else i for i in predictions]
        predictions = [1 if i > 1 else i for i in predictions]
        print(f'Prediction corrected:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
        lst_prediction.append(predictions)
        lst_models_trained.append(model)
        i+=1
    
    df_predictions = pd.DataFrame({
        'date': test_dates,
        'wp1': lst_prediction[0],
        'wp2': lst_prediction[1],
        'wp3': lst_prediction[2],
        'wp4': lst_prediction[3],
        'wp5': lst_prediction[4],
        'wp6': lst_prediction[5],        
    })
    return df_predictions, lst_models_trained

In [91]:
def make_submission_file_nmodels(lst_X_trains, lst_y_trains,lst_y_vmfs,lst_tests, lst_models, dates):
    i = 1
    lst_prediction = []
    lst_models_trained = []
    for X, y, test, model in zip(lst_X_trains, lst_y_vmfs, lst_tests, lst_models):
        print(f'--------------Model {i}--------------')
        for n in range(len(model)):
            model[n].fit(X,y[n])
        
        print(f'True:\n\tMin:{min(lst_y_trains[i-1])}\n\tMax:{max(lst_y_trains[i-1])}\n\tMean:{lst_y_trains[i-1].mean()}')
        
        for m in range(len(model)):
            if m==0:
                predictions = model[m].predict(test)
            else:
                predictions += model[m].predict(test)
        print(f'Prediction:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
#         predictions = [min(y) if i < 0 else i for i in predictions]
#         predictions = [max(y) if i > max(y) else i for i in predictions]
        predictions = [0 if i < 0 else i for i in predictions]
        predictions = [1 if i > 1 else i for i in predictions]
        print(f'Prediction corrected:\n\tMin:{min(predictions)}\n\tMax:{max(predictions)}\n\tMean:{np.mean(predictions)}')
        lst_prediction.append(predictions)
        lst_models_trained.append(model)
        i+=1
    
    df_predictions = pd.DataFrame({
        'date': test_dates,
        'wp1': lst_prediction[0],
        'wp2': lst_prediction[1],
        'wp3': lst_prediction[2],
        'wp4': lst_prediction[3],
        'wp5': lst_prediction[4],
        'wp6': lst_prediction[5],        
    })
    return df_predictions, lst_models_trained

## Submission 

In [ ]:
model_1=[]

for i in range(4):
    model=Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params_1[i]))])
    model_1+=[model]
    

In [ ]:
model_2=[]

for i in range(4):
    model=Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params_2[i]))])
    model_2+=[model]
    

In [ ]:
model_3=[]

for i in range(4):
    model=Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params_3[i]))])
    model_3+=[model]
    

In [ ]:
model_4=[]

for i in range(4):
    model=Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params_4[i]))])
    model_4+=[model]
    

In [ ]:
model_5=[]

for i in range(4):
    model=Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params_5[i]))])
    model_5+=[model]
    

In [ ]:
model_6=[]

for i in range(4):
    model=Pipeline([('scaler', MaxAbsScaler()),('ridge', LGBMRegressor(**params_6[i]))])
    model_6+=[model]
    

In [ ]:
y_vmf1=[]
y_vmf2=[]
y_vmf3=[]
y_vmf4=[]
y_vmf5=[]
y_vmf6=[]
for i in range(1,k+1):
    y_vmf1.append(vmf_1['IMFwp'+str(i)])
    y_vmf2.append(vmf_2['IMFwp'+str(i)])
    y_vmf3.append(vmf_3['IMFwp'+str(i)])
    y_vmf4.append(vmf_4['IMFwp'+str(i)])
    y_vmf5.append(vmf_5['IMFwp'+str(i)])
    y_vmf6.append(vmf_6['IMFwp'+str(i)])

In [ ]:
# model_1 = LGBMRegressor(**params_1)
# model_2 = LGBMRegressor(**params_2)
# model_3 = LGBMRegressor(**params_3)
# model_4 = LGBMRegressor(**params_4)
# model_5 = LGBMRegressor(**params_5)
# model_6 = LGBMRegressor(**params_6)

lst_models = [model_1, model_2, model_3, model_4, model_5, model_6]
#lst_params_none=[ , , , ]
#lst_models=[lst_params_none,lst_params_none,lst_params_none,lst_params_none,lst_params_none,lst_params_none]
lst_X_trains = [X1, X2, X3, X4, X5, X6]
lst_y_trains = [y1, y2, y3, y4, y5, y6]
lst_y_vmf=[y_vmf1, y_vmf2, y_vmf3, y_vmf4, y_vmf5, y_vmf6]

In [ ]:
lst_tests = []
for test in [test_wp1, test_wp2, test_wp3, test_wp4, test_wp5, test_wp6]:
    test = make_prediction_dataset(test)
    lst_tests.append(test)

In [ ]:
#df_predictions, lst_models_trained = make_submission_file(lst_X_trains, lst_y_trains, lst_y_vmf,lst_tests, lst_models, test_dates)

In [ ]:
df_predictions, lst_models_trained = make_submission_file_nmodels(lst_X_trains, lst_y_trains, lst_y_vmf,lst_tests, lst_models, test_dates)

## Saving models

In [ ]:
nb_sub = 41
model = "robustscaler-lgbm-vmf-cv"
prepro = 'RobustScaler'
postpro = "Prediction limited by 0-1"

In [ ]:
# df_predictions.to_csv('Predictions/submission_nb_10_full_maxabs-lgbm-featselect.csv', index=False, sep=';')
df_predictions.to_csv(f'Predictions/submission_nb_{nb_sub}_{model}.csv', index=False, sep=';')

In [ ]:
df_predictions.head()